In [1]:
import tensorflow as tf
import numpy as np
from functions import RNN_Train_Forecast,createGraphRNN2,RNN_generator,init_state_update,\
RNN_forecast_Repeat,init_state_update_LSTM,RNN_forecast_Repeat_LSTM
import cPickle

In [2]:
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']

In [3]:
learningRate = 1e-4
epoch = 25
repeat = 10
SavePath = '/home/will/Desktop/Neural Network/Sales Forecasting/final'

In [4]:
cardinalitys_X = [55, 4001, 34, 337, 2, 23, 17, 6, 18]
cardinalitys_T = [7, 2, 13, 2, 7, 4, 25, 2, 3]
dimentions_X = [2, 20, 1, 2, 1, 1, 1, 1, 1]
dimentions_T = [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [5]:
dX = sum(dimentions_X)
dT = sum(dimentions_T)
d = dX + dT + 2 # 2 for two cont variables

In [6]:
prefix = 'train_final'
y_np = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [7]:
prefix = 'test_SI_final'
y_np_val = np.loadtxt(prefix+'_Y',dtype=np.int32, delimiter=",") 
Con_np_val = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np_val = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Dis_np_val = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [8]:
index = np.loadtxt('Index_final',dtype=np.int32,delimiter=",") 

In [9]:
fixedPara = {'cardinalitys_X':cardinalitys_X,'cardinalitys_T':cardinalitys_T,'dimentions_X':dimentions_X\
             ,'dimentions_T':dimentions_T,'dX':dX,'d':d}

In [10]:
model_para_list = [{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.628044403851,'n_layers':2,'grad_clip':2.99125114123,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'relu'},'downsample':0.921508935737},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.813682171767,'n_layers':2,'grad_clip':4.71494036186,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.771180461646},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.702969880055,'n_layers':2,'grad_clip':2.01736824207,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.898455532427},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.428501799036,'n_layers':3,'grad_clip':4.2006247805,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.734974933319},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.459696006212,'n_layers':5,'grad_clip':5.76707667157,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.644960742075},
{'model_para':{'batch_size':50,'seq_len':32,'keep_prob':0.427920301113,'n_layers':4,'grad_clip':6.37998154905,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'relu'},'downsample':0.467438862634},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.716917304571,'n_layers':2,'grad_clip':1.7676565249,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.940803705956},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.677708118391,'n_layers':3,'grad_clip':1.0014287617,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.810976906715},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.801469391508,'n_layers':5,'grad_clip':2.86814589654,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.719331346325},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.819993479712,'n_layers':2,'grad_clip':4.74376042578,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.782327028395},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.415802367601,'n_layers':4,'grad_clip':6.38154110535,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.684181659341},
{'model_para':{'batch_size':100,'seq_len':16,'keep_prob':0.670033419799,'n_layers':2,'grad_clip':1.40163749265,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.82368610599},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.524084618388,'n_layers':3,'grad_clip':7.3201240237,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'relu'},'downsample':0.520909052689},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.745017883177,'n_layers':2,'grad_clip':7.89519173494,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.902410343017},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.400749504666,'n_layers':4,'grad_clip':4.14237357172,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'relu'},'downsample':0.913470970128},
{'model_para':{'batch_size':50,'seq_len':32,'keep_prob':0.897205518497,'n_layers':3,'grad_clip':4.95813489272,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'relu'},'downsample':0.583659186288},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.64949104808,'n_layers':2,'grad_clip':2.23405885747,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.670332198251},
{'model_para':{'batch_size':50,'seq_len':32,'keep_prob':0.732932282822,'n_layers':2,'grad_clip':1.32638769648,'cell_type': 'highway','optimizer':'Adam','actFun':'tanh'},'downsample':0.694517667059},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.464332365322,'n_layers':2,'grad_clip':4.33557727242,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.941907479812},
{'model_para':{'batch_size':25,'seq_len':64,'keep_prob':0.877049685508,'n_layers':3,'grad_clip':6.13619058959,'cell_type': 'NormLSTM','optimizer':'Adam','actFun':'tanh'},'downsample':0.884550903377},
]

In [13]:
y_SI = 0
for paras in model_para_list:
    y_SI = y_SI + RNN_Train_Forecast(paras,fixedPara,learningRate,epoch,SavePath,repeat,\
                                     y_np, weight_np,Con_np,Dis_np,X_np,Count_np,index,\
                                     Con_np_val,X_np_val,Dis_np_val,d)
y_SI = y_SI/len(model_para_list)

INFO:tensorflow:Restoring parameters from /home/will/Desktop/Neural Network/Sales Forecasting/final/grad_clip:2.99125114123-cell_type:NormLSTM-optimizer:Adam-actFun:relu-seq_len:64-n_layers:2-keep_prob:0.628044403851-batch_size:25-downsample:0.921508935737
INFO:tensorflow:Restoring parameters from /home/will/Desktop/Neural Network/Sales Forecasting/final/grad_clip:4.71494036186-cell_type:highway-optimizer:Adam-actFun:tanh-seq_len:64-n_layers:2-keep_prob:0.813682171767-batch_size:25-downsample:0.771180461646
INFO:tensorflow:Restoring parameters from /home/will/Desktop/Neural Network/Sales Forecasting/final/grad_clip:2.01736824207-cell_type:NormLSTM-optimizer:Adam-actFun:tanh-seq_len:64-n_layers:2-keep_prob:0.702969880055-batch_size:25-downsample:0.898455532427
INFO:tensorflow:Restoring parameters from /home/will/Desktop/Neural Network/Sales Forecasting/final/grad_clip:4.2006247805-cell_type:NormLSTM-optimizer:Adam-actFun:tanh-seq_len:64-n_layers:3-keep_prob:0.428501799036-batch_size:25-

In [15]:
import pandas as pd
pd.DataFrame({'id':y_np_val.flatten(),'unit_sales':y_SI.flatten()})\
  .to_csv('FinalSI.csv',index=False)

In [ ]:
newItem = pd.read_csv('newItem.csv',dtype={'id': 'int32','unit_sales': 'float32'})

In [ ]:
SI = pd.DataFrame({'id':y_np_SI.flatten(),'unit_sales':y_SI.flatten()})  

In [ ]:
zeroSubmission = pd.read_csv('zeroSubmission_final.csv',dtype={'id': 'int32','unit_sales': 'float32'})

In [ ]:
pd.concat([SI,newItem,zeroSubmission]).sort_values(['id']).to_csv(SavePath+'FinalEnsembleWoSecondStage.csv',index=False)